In [1]:
import pandas as pd
import yfinance as yf
import datetime
import backtrader as bt


In [2]:

cerebro = bt.Cerebro()
start_portfolio_value = cerebro.broker.getvalue()
print(f"start portfolio value: {start_portfolio_value}")


df_btc = yf.download("BTC",start= "2022-01-01", end="2023-01-01")
df_eth = yf.download("ETH-USD",start= "2022-01-01", end="2023-01-01" )
print(df_btc)
print(df_eth)

start portfolio value: 10000.0
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
                 Open       High        Low      Close  Adj Close  Volume
Date                                                                     
2022-01-03  98.309998  98.309998  98.294701  98.294701  95.546432     177
2022-01-04  98.279999  98.291496  98.279999  98.291496  95.543312  356148
2022-01-05  98.250000  98.269997  98.090103  98.096397  95.353676  356863
2022-01-06  97.985001  97.985001  97.985001  97.985001  95.245377     104
2022-01-07  97.894600  97.894600  97.894600  97.894600  95.157509       3
...               ...        ...        ...        ...        ...     ...
2022-12-23  90.771004  90.804497  90.760002  90.804497  89.343491    1308
2022-12-27  90.509598  90.509598  90.509598  90.509598  89.053345      15
2022-12-28  90.420998  90.434700  90.420998  90.434700  88.979645     168
2022-12-29  90.59

In [3]:
df_signals = pd.read_csv("/workspaces/projects/Backtesting/signals.csv")
new_df = df_signals.set_index('timestamp')['signal'].to_frame()
new_df.index = pd.to_datetime(new_df.index)
print(new_df.tail())
print(new_df.info())

                       signal
timestamp                    
2022-12-30 08:00:00 -0.253285
2022-12-30 16:00:00 -0.145851
2022-12-31 00:00:00 -0.140179
2022-12-31 08:00:00  0.142683
2022-12-31 16:00:00  0.339424
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1095 entries, 2022-01-01 00:00:00 to 2022-12-31 16:00:00
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   signal  1076 non-null   float64
dtypes: float64(1)
memory usage: 17.1 KB
None


In [4]:
class TestStrategy(bt.Strategy):
    def __init__(self):
        self.btc_price = self.data0.Close
        self.eth_price = self.data1.Close
        self.signal = new_df  

    def next(self):
        current_signal = self.signal.loc[self.datetime.datetime()]  # Access the signal for the current date
        print(f"Current Signal: {current_signal}")
        for idx, value in self.signal.iterrows():
            if value['signal'] > 1: 
                self.buy(data=self.data0,size=1)
                self.sell(data=self.data1,size=1)
            # elif value['signal'] < -1: 
            #     self.sell(data=self.data0)
            #     self.buy(data=self.data1)
            elif abs(value['signal']) < 0.3:
                self.close()
    

if __name__ == '__main__':
    cerebro = bt.Cerebro()
    
    btc = bt.feeds.PandasData(dataname = df_btc)
    cerebro.adddata(btc)
    
    eth = bt.feeds.PandasData(dataname = df_eth)
    cerebro.adddata(eth)
    # signals = bt.feeds.PandasData(dataname = new_df)
    # cerebro.adddata(signals)
    
    cerebro.addstrategy(TestStrategy)  # Adding the test strategy
    
    cerebro.addanalyzer(bt.analyzers.Returns)
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.0)
    cerebro.addanalyzer(bt.analyzers.DrawDown)
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer)
    
    results = cerebro.run()
    
    returns = results[0].analyzers.returns.get_analysis()
    sharpe_ratio = results[0].analyzers.sharperatio.get_analysis()
    drawdown_info = results[0].analyzers.drawdown.get_analysis()
    trade_info = results[0].analyzers.tradeanalyzer.get_analysis()

    print("Returns:", returns)
    print("Sharpe Ratio:", sharpe_ratio)
    print("Drawdown Info:", drawdown_info)
    print("Trade Info:", trade_info)

Current Signal: signal   NaN
Name: 2022-01-03 00:00:00, dtype: float64
Current Signal: signal   NaN
Name: 2022-01-04 00:00:00, dtype: float64
Current Signal: signal   NaN
Name: 2022-01-05 00:00:00, dtype: float64
Current Signal: signal   NaN
Name: 2022-01-06 00:00:00, dtype: float64
Current Signal: signal   NaN
Name: 2022-01-07 00:00:00, dtype: float64
Current Signal: signal   -1.393154
Name: 2022-01-08 00:00:00, dtype: float64
Current Signal: signal   -1.283257
Name: 2022-01-09 00:00:00, dtype: float64
Current Signal: signal   -1.185018
Name: 2022-01-10 00:00:00, dtype: float64
Current Signal: signal   -0.917038
Name: 2022-01-11 00:00:00, dtype: float64
Current Signal: signal   -0.585802
Name: 2022-01-12 00:00:00, dtype: float64
Current Signal: signal    0.502872
Name: 2022-01-13 00:00:00, dtype: float64
Current Signal: signal    0.816032
Name: 2022-01-14 00:00:00, dtype: float64
Current Signal: signal    0.838862
Name: 2022-01-15 00:00:00, dtype: float64
Current Signal: signal    0.9

In [5]:
final_portfolio_value = cerebro.broker.getvalue()
print(f"Final portfolio value: {final_portfolio_value}")
print(results)

Final portfolio value: 5662491329.784424
